In [0]:
# Crea un RDD de ejemplo
data = [1, 2, 3, 4, 5]
rdd = spark.sparkContext.parallelize(data)
# Transformación map(func)
mapped_rdd = rdd.map(lambda x: x * 2)
print("map(func):", mapped_rdd.collect())b

map(func): [2, 4, 6, 8, 10]


In [0]:
# Transformación filter(func)
filtered_rdd = rdd.filter(lambda x: x % 2 == 0)
print("filter(func):", filtered_rdd.collect())

filter(func): [2, 4]


In [0]:
# Crear un RDD de pares clave-valor para las transformaciones restantes
key_value_rdd = rdd.map(lambda x: (x, x))
print(key_value_rdd)

PythonRDD[3] at RDD at PythonRDD.scala:58


In [0]:
# Transformación groupByKey()
grouped_rdd = key_value_rdd.groupByKey()
print("groupByKey():", grouped_rdd.mapValues(list).collect())

groupByKey(): [(1, [1]), (2, [2]), (3, [3]), (4, [4]), (5, [5])]


In [0]:
# Transformación reduceByKey(func)
reduced_rdd = key_value_rdd.reduceByKey(lambda x, y: x + y)
print("reduceByKey(func):", reduced_rdd.collect())

reduceByKey(func): [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]


In [0]:
# Transformación sortByKey()
sorted_rdd = key_value_rdd.sortByKey(ascending=False)
print("sortByKey():", sorted_rdd.collect())

sortByKey(): [(5, 5), (4, 4), (3, 3), (2, 2), (1, 1)]


In [0]:
# Transformación join(otherDataset)
other_rdd = spark.sparkContext.parallelize([(1, 'A'), (2, 'B')])
joined_rdd = key_value_rdd.join(other_rdd)
print("join(otherDataset):", joined_rdd.collect())

join(otherDataset): [(1, (1, 'A')), (2, (2, 'B'))]


In [0]:
# Transformación union(otherDataset)
other_data = [6, 7, 8]
other_rdd = spark.sparkContext.parallelize(other_data)
unioned_rdd = rdd.union(other_rdd)
print("union(otherDataset):", unioned_rdd.collect())

union(otherDataset): [1, 2, 3, 4, 5, 6, 7, 8]


In [0]:
# Transformación distinct()
duplicated_data = [1, 2, 2, 3, 3, 4, 5, 5]
duplicated_rdd = spark.sparkContext.parallelize(duplicated_data)
distinct_rdd = duplicated_rdd.distinct()
print("distinct():", distinct_rdd.collect())

distinct(): [1, 2, 3, 4, 5]


In [0]:
from pyspark.sql import SparkSession
import requests
import pandas as pd
import io
import warnings
warnings.filterwarnings("ignore")

In [0]:
# Creating spar session object
spark = SparkSession.builder.appName('manipulacion_data').getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
df_url = "https://raw.githubusercontent.com/narencastellon/Mi-Dataset/refs/heads/main/diabetes_data.csv"
# Download the CSV file using requests
response = requests.get(df_url)
response.raise_for_status()  # Raise an exception for bad responses
# Read the CSV data into a Pandas DataFrame
diabetes = pd.read_csv(io.StringIO(response.text), sep=',')
# Convert the Pandas DataFrame to a Spark DataFrame
df = spark.createDataFrame(diabetes)
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [0]:
df.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



In [0]:
df.columns

Out[15]: ['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [0]:
print(df.count(), len(df.columns))

768 9


In [0]:
df.printSchema()

root
 |-- Pregnancies: long (nullable = true)
 |-- Glucose: long (nullable = true)
 |-- BloodPressure: long (nullable = true)
 |-- SkinThickness: long (nullable = true)
 |-- Insulin: long (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: long (nullable = true)
 |-- Outcome: long (nullable = true)



In [0]:
df.describe().show(truncate=5)

+-------+-----------+-------+-------------+-------------+-------+-----+------------------------+-----+-------+
|summary|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|  BMI|DiabetesPedigreeFunction|  Age|Outcome|
+-------+-----------+-------+-------------+-------------+-------+-----+------------------------+-----+-------+
|  count|        768|    768|          768|          768|    768|  768|                     768|  768|    768|
|   mean|      3....|  12...|        69...|        20...|  79...|31...|                   0....|33...|  0....|
|  st...|      3....|  31...|        19...|        15...|  11...|7....|                   0....|11...|  0....|
|    min|          0|      0|            0|            0|      0|  0.0|                   0.078|   21|      0|
|    max|         17|    199|          122|           99|    846| 67.1|                    2.42|   81|      1|
+-------+-----------+-------+-------------+-------------+-------+-----+------------------------+-----+-------+



In [0]:
df.select('Pregnancies', 'Age').show(5)

+-----------+---+
|Pregnancies|Age|
+-----------+---+
|          6| 50|
|          1| 31|
|          8| 32|
|          1| 21|
|          0| 33|
+-----------+---+
only showing top 5 rows



In [0]:
import pyspark.sql.functions as F
df.select(F.col('Pregnancies'), F.col('Age')).show(5)

+-----------+---+
|Pregnancies|Age|
+-----------+---+
|          6| 50|
|          1| 31|
|          8| 32|
|          1| 21|
|          0| 33|
+-----------+---+
only showing top 5 rows



In [0]:
df.filter(df['Age']<40).show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



In [0]:
df.filter(df['Age']<40).select('Insulin','Outcome').show(3)

+-------+-------+
|Insulin|Outcome|
+-------+-------+
|      0|      0|
|      0|      1|
|     94|      0|
+-------+-------+
only showing top 3 rows



In [0]:
df.filter(df['age']>60).filter(df['Outcome']=='1').show(3)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          4|    146|           92|            0|      0|31.2|                   0.539| 61|      1|
|          0|    105|           84|            0|      0|27.9|                   0.741| 62|      1|
|          2|    158|           90|            0|      0|31.6|                   0.805| 66|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 3 rows



In [0]:
df.filter( (df['Outcome']==1) & (df['Pregnancies']>9)).show(3)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|         10|    168|           74|            0|      0|38.0|                   0.537| 34|      1|
|         11|    143|           94|           33|    146|36.6|                   0.254| 51|      1|
|         10|    125|           70|           26|    115|31.1|                   0.205| 41|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 3 rows



In [0]:
df.filter(df['age']>40).count()

Out[31]: 194

In [0]:
df.where((df['Outcome']==1)&(df['Pregnancies']>9)).show(3)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|         10|    168|           74|            0|      0|38.0|                   0.537| 34|      1|
|         11|    143|           94|           33|    146|36.6|                   0.254| 51|      1|
|         10|    125|           70|           26|    115|31.1|                   0.205| 41|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 3 rows



In [0]:
df.where(df['age']>40).count()

Out[33]: 194

In [0]:
df.withColumn('New_Age', df['age']+10).show(3)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|New_Age|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|     60|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|     41|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|     42|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+-------+
only showing top 3 rows



In [0]:
df.groupBy('Outcome').count().show()

+-------+-----+
|Outcome|count|
+-------+-----+
|      0|  500|
|      1|  268|
+-------+-----+



In [0]:
df.select('Pregnancies').distinct().count()

Out[36]: 17

In [0]:
df.groupBy('Outcome').agg(F.mean('age').alias('age_mean')).show()

+-------+-----------------+
|Outcome|         age_mean|
+-------+-----------------+
|      0|            31.19|
|      1|37.06716417910448|
+-------+-----------------+



In [0]:
def diabetes(case):
    if case==1:
        return 'diabetes'
    else:
        return 'no diabetes'

In [0]:
#Applying User-Defined Functions
from pyspark.sql.types import *
diabetes_udf= F.udf(diabetes, StringType())
df.withColumn('diabete_case', diabetes_udf(df['Outcome'])).show(3)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|diabete_case|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|    diabetes|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0| no diabetes|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|    diabetes|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------------+
only showing top 3 rows



In [0]:
df.drop('Insulin').show(3)

+-----------+-------+-------------+-------------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+----+------------------------+---+-------+
|          6|    148|           72|           35|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|23.3|                   0.672| 32|      1|
+-----------+-------+-------------+-------------+----+------------------------+---+-------+
only showing top 3 rows



In [0]:
print('The number of records: ',df.count() )
df=df.dropDuplicates()
print('The number of records after removing the duplicate: ', df.count())

The number of records:  768
The number of records after removing the duplicate:  768


In [0]:
df.write.format('csv').option('header','true').save('./CursoSpark/my_dataset01.csv')

In [0]:
df.write.csv('./my_dataset.csv')
print('ok')

ok
